In [16]:
# imports
import time
from IPython.display import clear_output
from tqdm.notebook import tqdm
import json
import re
import os
import numpy as np
from PIL import Image
from sklearn.cluster import MeanShift, estimate_bandwidth
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import io

In [17]:
# constants _________
# params
BLOCK_MARGIN = 20
QUANTILE_VAL_BLOCK_FORMING = 0.1 #значение квантиля при кластеризации отступов строк в тексте

# lists with data
pages_with_blocks = []
names_of_pages = []

# blocks data
final_blocks = []
fnames = []
regnums = []
ords = []

In [3]:
# MAIN TEST

# imgs = r'/work/s3fs/letopisi/1962/32/0034.jpg'

# yandex_paths = ['/work/s3fs/letopisi/1962/32_yandex_ocr/0034.jpg.json']
# lilt_paths = ['/work/s3fs/letopisi/1962/32_lilt_res/0034.jpg.json']
# img_path = '/work/s3fs/letopisi/1962/32/0034.jpg'

# length = len(yandex_paths)

In [18]:
# MAIN TEST 2
imgs = r'/work/s3fs/letopisi/1972/52/0041.jpg'

yandex_paths = ['/work/s3fs/letopisi/1972/52_yandex_ocr/0041.jpg.json']
lilt_paths = ['/work/s3fs/letopisi/1972/52_lilt_res/0041.jpg.json']

img_path = '/work/s3fs/letopisi/1972/52/0041.jpg'

length = len(yandex_paths)

In [ ]:
# functions
def opener(file):
    with open(file, 'r') as f:
        open_file = f.read().split('\n')
    return open_file

# get yandex json paths from imgs paths
def get_yandex_json_path(img_paths):
    yandjson_path_list =  []
    for i in img_paths:
        nomad = i.split('/')
        # print(len(nomad))
        # print(nomad)
        if len(nomad) == 7:
            yandex_str = nomad[5] + '_yandex_ocr'
            yandex_str_2 = nomad[6] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] +
                                '/' + yandex_str + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
        if len(nomad) == 8:
            yandex_str = nomad[5] + '_yandex_ocr'
            yandex_str_2 = nomad[7] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] + '/' + yandex_str +
                                '/' + nomad[6] + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
        if len(nomad) == 9:
            yandex_str = nomad[5] + '_yandex_ocr'
            yandex_str_2 = nomad[8] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] + '/' + yandex_str +
                                '/' + nomad[6] + '/' + nomad[7] + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
    return yandjson_path_list
 
# get lilt (tesseract) json paths from imgs paths
def get_lilt_json_path(img_paths):
    yandjson_path_list =  []
    for i in img_paths:
        nomad = i.split('/')
        # print(len(nomad))
        # print(nomad)
        if len(nomad) == 7:
            yandex_str = nomad[5] + '_lilt_res'
            yandex_str_2 = nomad[6] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] +
                                '/' + yandex_str + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
        if len(nomad) == 8:
            yandex_str = nomad[5] + '_lilt_res'
            yandex_str_2 = nomad[7] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] + '/' + yandex_str +
                                '/' + nomad[6] + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
        if len(nomad) == 9:
            yandex_str = nomad[5] + '_lilt_res'
            yandex_str_2 = nomad[8] + '.json'
            final_json_path = str('/' + nomad[1] + '/' + nomad[2] + 
                                '/' + nomad[3] + '/' + nomad[4] + '/' + yandex_str +
                                '/' + nomad[6] + '/' + nomad[7] + '/' + yandex_str_2)
            yandjson_path_list.append(final_json_path)
    return yandjson_path_list

In [19]:
# MAIN CODE without correction
for yandex_ocr_json_path, json_path in tqdm(zip(yandex_paths, lilt_paths), total=length):    
    try:
        # yandex_ocr_json_path = json_path.replace('lilt_res', 'yandex_ocr')
        # with open(img_path, "rb") as f:
        #     files = {"file": f}
        #     response = requests.post(url, files=files)
        #     parsed = json.loads(response.text)

    #   распаковка json с яндексом
        with open(yandex_ocr_json_path, "r", encoding = 'utf-8') as f:
            data = f.read()
            parsed_yandex = json.loads(data)
    except Exception as e:
                    print(e)
    try:
        word_info = []
        for block in parsed_yandex['result']['results'][0]['results'][0]['textDetection']['pages'][0]['blocks']:
            for line in block['lines']:
                for word_payload in line['words']:
                    # print(word_payload)
                    text = word_payload['text']
                    vertices = word_payload['boundingBox']['vertices']
                    word_coord = (int(vertices[0]['x']), int(vertices[0]['y']), int(vertices[2]['x']), int(vertices[2]['y']))
                    word_info.append((word_coord, text))
    except Exception as e:
                    print(e)
        
    # print(word_info)
    try:
        with open(json_path, "r", encoding = 'utf-8') as f:
            data = f.read()
            parsed = json.loads(data)
        # img_path = json_path.replace(json_path.split('/')[5], json_path.split('/')[5].split('_')[0])[:-5]
        # print(img_path)
        im = Image.open(img_path)
        im_orig_size = im.size
        print(im_orig_size)
        im_resized = im.resize((384,384))
        # display(im_resized)
        # print(img_path, '\n', parsed['line_boxes'], '\n')
        stride_percentages = []
        # print(parsed['line_boxes']['0'])
        for box in parsed['line_boxes']['0']:
            stride_right_percentage = int(box[0] / im_orig_size[0] * 100)
            stride_percentages.append(box[0])
    except Exception as e:
                    print(e)
    # print(stride_percentages) #координаты начала строки
    try:
        X = list(zip(stride_percentages,np.zeros(len(stride_percentages))))
        print(X)
        bandwidth = estimate_bandwidth(X, quantile=QUANTILE_VAL_BLOCK_FORMING)
        ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
        ms.fit(X)
        labels = ms.labels_
        cluster_centers = ms.cluster_centers_


        labels_unique = np.unique(labels)
        print(labels_unique)
        n_clusters_ = len(labels_unique)

        coord_clusters = {}
        for k in range(n_clusters_):
            my_members = labels == k
            coord_clusters[k] = np.array(X)[my_members, 0]
            print ("cluster {0}: {1}".format(k, np.array(X)[my_members, 0]))

        block = []
        x0s = []
        y0s = []
        x1s = []
        y1s = []
        count_blocks = 1
    except Exception as e:
                    print(e)
    
    # print(coord_clusters)
    try:
        page_blocks_texts = []
        for box in parsed['line_boxes']['0']:
            cropped = im.crop((box[0], box[1], box[2], box[3]))
            cropped = cropped.resize((1024,224))
            # display(cropped)
            # continue
            
            if box[0] not in coord_clusters[0]:
                block_np_array = np.concatenate([block], 0)
                # print(block_np_array)
                # print(block_np_array.shape)
                if block_np_array.shape == (0,):
                    x0s.append(box[0])
                    y0s.append(box[1])
                    x1s.append(box[2])
                    y1s.append(box[3])
                    block.append(box)
                    continue
                # top left
                min_x0 = min(x0s) - BLOCK_MARGIN
                min_y0 = min(y0s) - BLOCK_MARGIN
                # bottom right
                max_x1 = max(x1s) + BLOCK_MARGIN
                max_y1 = max(y1s) + BLOCK_MARGIN
                # print((min_x0, min_y0, max_x1, max_y1))
                try:
                    if min(x0s) in coord_clusters[0] or min(x0s) in coord_clusters[1]:
                        words_in_block = []
                        for word_payload in word_info:
                            word_coords = word_payload[0]
                            if (word_coords[0] > min_x0 and word_coords[2] < max_x1) and (word_coords[1] > min_y0 and word_coords[3] < max_y1):
                                words_in_block.append(word_payload[1])
                        text_in_block = ' '.join(words_in_block)
                        # cropped = im.crop((min_x0, min_y0, max_x1, max_y1))
                        # cropped = cropped.resize((1024,224))
                        # display(cropped)
                        # print(text_in_block)
                        page_blocks_texts.append(text_in_block)
                        # print(page_blocks_texts)
                except Exception as e:
                    print(e)
                    continue
                print(page_blocks_texts)
                block = []
                x0s = []
                y0s = []
                x1s = []
                y1s = []

                count_blocks += 1
                x0s.append(box[0])
                y0s.append(box[1])
                x1s.append(box[2])
                y1s.append(box[3])
                block.append(box)
            else:
                x0s.append(box[0])
                y0s.append(box[1])
                x1s.append(box[2])
                y1s.append(box[3])
                block.append(box)
        pages_with_blocks.append((img_path,page_blocks_texts))
    except Exception as e:
        # with open('1922_1926.log', 'a') as f:
        #     f.write(img_path+'\n')
        print('problem with', img_path, 'Error:', e)
        continue

  0%|          | 0/1 [00:00<?, ?it/s]

(1578, 2454)
[(218, 0.0), (147, 0.0), (154, 0.0), (157, 0.0), (217, 0.0), (146, 0.0), (154, 0.0), (149, 0.0), (147, 0.0), (220, 0.0), (149, 0.0), (149, 0.0), (152, 0.0), (218, 0.0), (149, 0.0), (149, 0.0), (154, 0.0), (218, 0.0), (158, 0.0), (151, 0.0), (152, 0.0), (151, 0.0), (212, 0.0), (151, 0.0), (151, 0.0), (1197, 0.0), (220, 0.0), (149, 0.0), (152, 0.0), (149, 0.0), (149, 0.0), (149, 0.0), (1074, 0.0), (380, 0.0), (220, 0.0), (149, 0.0), (151, 0.0), (157, 0.0), (220, 0.0), (157, 0.0), (154, 0.0), (151, 0.0), (141, 0.0), (221, 0.0), (151, 0.0), (149, 0.0), (152, 0.0), (221, 0.0), (151, 0.0), (152, 0.0), (152, 0.0), (160, 0.0), (1411, 0.0)]
[0 1 2 3 4]
cluster 0: [218. 147. 154. 157. 217. 146. 154. 149. 147. 220. 149. 149. 152. 218.
 149. 149. 154. 218. 158. 151. 152. 151. 212. 151. 151. 220. 149. 152.
 149. 149. 149. 220. 149. 151. 157. 220. 157. 154. 151. 141. 221. 151.
 149. 152. 221. 151. 152. 152. 160.]
cluster 1: [1411.]
cluster 2: [1197.]
cluster 3: [1074.]
cluster 4: [380.]

In [8]:
print(pages_with_blocks)

[('C:/20122023block_split/img_sample/work/s3fs/letopisi/1962/32/0034.jpg', ['(Ин-т орган. химин АН УССР. Кневский гос. ун-т им. Т. Г. Шевченко). 100 экз. 12 к. - [ 62-42127 ] п 541.14', 'Химия и химическая технология. 2. - На литов. и рус. яз. См. 27655', '27828. Долбилин, И. и Удилов, В. Механизация и автоматизация на лесозаготовках. Свердловск, Кн. изд., 1962. 99 с. с илл. 20 см. 3.000 экз. 15 к. -- [ 62-47173 ] п 634.98.0023 (47.811)', '27829. Жуков, Е. и Коршун, Л. Терпеноколлоксилиновые лаки для от- делки древесины. Вильнюс, Респ. ин-т науч.-техн. информации и пропа- ганды, 1962. 20 с. с черт. 20 см. (Гос. ком. Совета Министров Литов. ССР по координации науч.-неслед. работ). 60 экз. 20 к. На литов. яз.- [ 62-42426 ] п 674.02 : 667.67-667.67', '27830. Колосов, А. Директор леспромхоза [П. Д. Брагин. Падан. лес- промхоз]. Петрозаводск, Карел. кн. изд., 1962. 39 с. с портр. 17 см. 3.000 экз. 2 к. --- [ 62-46918 ] п 634.98ст (47.20)', '27831. Реутов, Ю. М. Тракторы на зимней сплотке др

In [14]:
pages_with_blocks[1][1][3]


'27829. Жуков, Е. и Коршун, Л. Терпеноколлоксилиновые лаки для от- делки древесины. Вильнюс, Респ. ин-т науч.-техн. информации и пропа- ганды, 1962. 20 с. с черт. 20 см. (Гос. ком. Совета Министров Литов. ССР по координации науч.-неслед. работ). 60 экз. 20 к. На литов. яз.- [ 62-42426 ] п 674.02 : 667.67-667.67'

In [6]:
# если блок в pages_with_blocks начинается не с порядкового номера, он добавляется к предыдущей строке
                          
regex_pattern = r'(^\d+\. [А-Я])'

def correct_blocks(pages_with_blocks):
    correct_blocks = []
    for p in pages_with_blocks:
        info = p[1]
        for idx, block_uni in enumerate(info):
            # print(block_uni)
            str_with_ords = re.search(regex_pattern, block_uni, re.IGNORECASE)
            if str_with_ords:
                correct_blocks.append(block_uni)
            else:
                # print(block_uni, idx)
                correct_blocks.append(info[idx - 1] + ' ' + block_uni)
                if idx >= 1:
                    del correct_blocks[idx-1]
    return p[0], correct_blocks


In [7]:
# соединяем блоки без порядкового номера с вышестоящими по порядку блоками
correct_blocks(pages_with_blocks)

('C:/20122023block_split/img_sample/work/s3fs/letopisi/1962/32/0034.jpg',
 ['(Ин-т орган. химин АН УССР. Кневский гос. ун-т им. Т. Г. Шевченко). 100 экз. 12 к. - [ 62-42127 ] п 541.14 Химия и химическая технология. 2. - На литов. и рус. яз. См. 27655',
  '27828. Долбилин, И. и Удилов, В. Механизация и автоматизация на лесозаготовках. Свердловск, Кн. изд., 1962. 99 с. с илл. 20 см. 3.000 экз. 15 к. -- [ 62-47173 ] п 634.98.0023 (47.811)',
  '27829. Жуков, Е. и Коршун, Л. Терпеноколлоксилиновые лаки для от- делки древесины. Вильнюс, Респ. ин-т науч.-техн. информации и пропа- ганды, 1962. 20 с. с черт. 20 см. (Гос. ком. Совета Министров Литов. ССР по координации науч.-неслед. работ). 60 экз. 20 к. На литов. яз.- [ 62-42426 ] п 674.02 : 667.67-667.67',
  '27830. Колосов, А. Директор леспромхоза [П. Д. Брагин. Падан. лес- промхоз]. Петрозаводск, Карел. кн. изд., 1962. 39 с. с портр. 17 см. 3.000 экз. 2 к. --- [ 62-46918 ] п 634.98ст (47.20)',
  '27831. Реутов, Ю. М. Тракторы на зимней сплот

In [ ]:
# get delimiters list
# if most of delimiters has the same lenght - continue, else - drop unusual delimiter
# check string on delimiters
# split string
# append string to new list for every page

ИСПРАВЛЕНИЕ ТЕКСТОВЫХ БЛОКОВ ПОСТФАКТУМ

In [15]:
from collections import Counter

In [16]:
# DEMO text

text_demo = ['42019. Мазанов М. Б. Защита плодовых культур от вредителей и бо- лезней в Кабардинс-Балкарии. Нальчик, «Эльбрус», 1972. 95 с. с ил.; 1 л. табл. (НТО сельск. хоз-ва КБАССР). - 17 см. 700 экз. 24 к. В пер. [ 72-82354 ] п 632.2./7 : 634.17.7 (470.64) 42020. Механизация трудоемких процессов в виноградарстве. Мате- риалы всесоюз. семинара 5-7 окт. 1971 г. [Ред. коллегия: Я. И. Потапенко (гл. ред.) и др.] Новочеркасск, 1972. 110 с. с ил. (МСХ РСФСР. Всерос. науч.-исслед. ин-т винеградарства и виноделия). Алф. указ. авт.: c. 109. -- 22 см. 500 экз. 50 к. - [ 72-82647 ] п 631.171 : 634.8 (063) 42021. Мироновские пшеницы. Под ред. В. Н. Ремесло. М., «Ко- лос», 1972. 288 с. с ил. (50-летию образования СССР). - На обороте тит. л. авт.: В. Н. Ремесло, М. А. Говорун, А. И. Суховецкий и др. - 21 см. 14.000 экз. 77 к. В пер. - [ 72-82174 ] п 633.1i«324» 42022. Производство, заготовка и хранение кормов. [Сборник статей. Отв. ред. В. И. Лисунов]. Ставрополь, 1972. 129 с. с черт. (М-во сельск. хоз-ва СССР. Ставроп. с.-х. ин-т. Науч. труды. Вып. 35. Т. 2) -- 20 см. 1,000 экз. 65 к. - [72-48814ж] 633.2/.4-1-636.085](470.63) 42023. Расы некоторых видов головни. Метод. указания по идентч- фикации. Л., 1972. 66 с. (ВАСХНИЛ. Всесоюз. науч.-исслед. ин-т расте- ниеводства им. Н. И. Вавилова). - Списки лит. в конце разделов. - На обороте тнт. л. авт.: В. И. Кривченко, Д. В. Мягкова, Л. Г. Щелко и др.-- 21 см. 500 экз. 40 к. - [ 72-81730 ] п 632.485.1.07 (083.93) 42024. Рычков Н. И., Олефир Е. П. Техника орошения садов и ягодни- ков. М., Россельхозиздат, 1972. 62 с. с ил. (Б-чка мелиоратора). -- На обл. авт. не указаны. - 20 см. 26.000 экз. 11 к. - [72-814801 п', '634.1/.7 : 631.67 42025. Цамутали А. С. Проблемы расширенного воспроизводства в хлопкосеющих колхозах Узбекистана. Ташкент, «Фан», 1972. 284 с. (М-во сельск. хоз-ва УзССР. Науч.-исслед. ин-т экономики сельск. хоз-ва). - Список лит.: с. 276-283 . - На пер. загл.: Расширенное воспро- изводство в хлопкосеющих колхозах. На корешке загл.: Расширенное вос- производство в колхозах. - 21 см. 1,000 экз. 1 р. 94 к. В пер. - [ 72-82106 ] п', '17.5 Лесное хозяйство. Лесохозяйственные науки 42026. Васильев П. В. Леса на службе социального прогресса. M., «Знание», 1972. 47 с. (Новое в жизни, науке, технике. Серия «Наука о Земле». 10). Список лит.: с. 46. - 21 см. 43.600 экз. 9 к. - [ 72-81488 ] п 634.0 42027. Вопросы селекции лесных и декоративных древесных растений. [Сборник статей. Ред. коллегия: А. Я. Любавская (отв. ред.) и др.] М., 1972. 96 с. с ил. (М-во высш. и сред. спец. образования СССР. Моск. лесо- техн. ин-т. Науч. труды. Вып. 43). - Списки лит. в конце статей. - 22 см. 700 экз. 43 к. - [72-48793ж] 634.0.165.6 + 635.977:631.527 42028. Вопросы экологии сосняков Севера. [Сборник статей. Ред. кол- легия: Л. А. Верхоланцева (отв. ред.) и др.] Сыктывкар, 1972. 135 с. с ил.; 3 л. ил. (АН СССР. Труды Коми филиала. № 24). Списки лит. в конце статей. - 27 см. 500 экз. 92 к. - [72-50144ж] 634.0.174.754 (47-17) 42029. Вопросы экономики лесного хозяйства Карельской АССР. [Сборник статей. Ред. коллегия: И. Я. Валентик (отв. ред.) и др.] Пет- розаводск, 1972. 168 с. с ил. (Карельск. филиал АН СССР. Отд. экономи- ки). - Список лит.: с. 161--165 (107 назв.) - 21 см. 700 экз. 70 к. [ 72-82640 ] п 634.0.6 (470.22)']


In [17]:
len(text_demo)

3

In [18]:
# functions

def get_delimiter(text_demo):

    re_split_pattern = r'(\d+\. [А-Я])'
    del_list = []

    for i in text_demo:
        # print(i)
        tag = re.findall(re_split_pattern, i) #.group()
        # print(tag)
        for t in tag:
            del_list.append(t)
    return del_list

def check_list_delimiter(delimiters):
    lenght = []
    for d in delimiters:
        lenght.append(len(d))
    return lenght   

In [24]:
def filter_delimiters(lenghs, delimiters):
    filter_delimiters = [] #пустой список для отфильтрованных разделителей
    for l, delim in zip(lenghs, delimiters):
        a = Counter(lenghs).most_common(1) #наиболее частотная длина разделителя (пор.номер + заглавная буква - автор или название книги)
        ln, count = a[0]
        if len(delim) == ln or len(delim) == ln - 1: #если длина разделителя соответствует среднему значению на данной странице или меньше среднего на 1 символ
            filter_delimiters.append(delim)
    return filter_delimiters

In [36]:
# def split_blocks(text_demo, filter_del):
    # ....


In [38]:
# MAIN
delimiters = get_delimiter(text_demo)
lenghs = check_list_delimiter(delimiters)
filter_del = filter_delimiters(lenghs, delimiters)
# in progress
# output_blocks = split_blocks(text_demo, filter_del)

print(delimiters)
# print(lenghs)
print(filter_del)

['42019. М', '42020. М', '42021. М', '42022. П', '35. Т', '42023. Р', '42024. Р', '42025. Ц', '42026. В', '42027. В', '42028. В', '42029. В']
['42019. М', '42020. М', '42021. М', '42022. П', '42023. Р', '42024. Р', '42025. Ц', '42026. В', '42027. В', '42028. В', '42029. В']


In [1]:
# string = "Маша, | Петя! ;Коля"
# delimiters = [",", "|", ";"]
 
# for delimiter in delimiters:
#     string = " ".join(string.split(delimiter))
 
# result = string.split()
 
# print(result)

['Маша', 'Петя!', 'Коля']


ВИЗУАЛИЗАЦИЯ КООРДИНАТ

In [ ]:
test_lilt_3 = ['/work/s3fs/letopisi/1977/указатель серийных изданий 1977г_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1965/доп вып вспомогательные указатели 4-6_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1973/41_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1974/дополнительный выпуск 9_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1968/38_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1969/6_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1966/доп вып вспомогательные указатели1-3_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1976/вспомогательные указатели 1_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1967/вспомогательные указатели 1_lilt_res/0003.jpg.json',
 '/work/s3fs/letopisi/1969/25_lilt_res/0003.jpg.json']

test_yandex_3 = []
for i in test_lilt_3:
    i_yand = i.replace('_lilt_res', '_yandex_ocr')
    test_yandex_3.append(i_yand)

In [ ]:
# __________ визуализация линий из координат в tesseract (lilt) json _________

# for yandex_ocr_json_path, json_path in tqdm(zip(yandex_paths, lilt_paths), total=length):  
for yandex_ocr_json_path, json_path in tqdm(zip(test_yandex_3, test_lilt_3), total=length):      
        # _______________ соотносим изображения и координаты линий + распознанные линии текста в Tesseract____________
    try:   
        with open(json_path, "r") as f:
            data = f.read()
            parsed = json.loads(data)
#             обозначаем имя изображения
        img_path = json_path.replace(json_path.split('/')[5], json_path.split('/')[5].split('_')[0])[:-5]
        # print(img_path)
        im = Image.open(img_path)
        im_orig_size = im.size
        # print(im_orig_size)
        im_resized = im.resize((384,384))
    #     # display(im_resized)
        # print(img_path, '\n', parsed['line_boxes'], '\n')
        stride_percentages = []
        # print(parsed['line_boxes']['0'])
        # for box in parsed['line_boxes']['0']:
        #     print(box)
        #         format for cv2 visualization
        im_cv = cv2.imread(img_path)
        for box in parsed['line_boxes']['0']:
            vis = cv2.rectangle(im_cv, (box[0], box[1]), (box[2], box[3]), (255, 0, 255), 2)
        i = img_path.split('/')
        # plt.imsave(f'img_examples_log/err_yand_24clust/{i[4]}__{i[5]}__{i[6]}', vis)
        # chronicles/img_examples_log/block_split_imgs - путь сохранения примеров
        print('OK')
    except Exception as err:
        print(err)

In [ ]:
# _____ визуализация блоков от yandex (координаты слов) ____
# for yandex_ocr_json_path, json_path in tqdm(zip(yandex_paths, lilt_paths), total=length):  
for yandex_ocr_json_path, json_path in tqdm(zip(test_yandex_3, test_lilt_3), total=length): 
    print(yandex_ocr_json_path)
    print(json_path)
    with open(yandex_ocr_json_path, "r") as f:
        data = f.read()
        # print(data)
    # with open('json_examples.txt', 'w') as f:
    #     f.write(f'{yandex_ocr_json_path}\n{json_path}\n{data}\n\n\n')
        parsed_yandex = json.loads(data) 
        # print(parsed_yandex)
        
    # ________ получение координат bbox у отдельных слов из json _______
    #     0 - левый верхний угол
    try:    
        word_info = []
        for block in parsed_yandex['result']['results'][0]['results'][0]['textDetection']['pages'][0]['blocks']:
            # print(block)
            for line in block['lines']:
                # print(line)
                for word_payload in line['words']:
                    # print(word_payload)
                    text = word_payload['text']
                    vertices = word_payload['boundingBox']['vertices']
                    word_coord = (int(vertices[0]['x']), int(vertices[0]['y']), int(vertices[2]['x']), int(vertices[2]['y']))
                    word_info.append((word_coord, text))
        # print(word_info) #список из bbox конкретного слова (кортеж 4 координаты) и строки, соответствующей слову
        # print(word_info[0])
    except Exception as err:
        print(err)
        print('ONE')
            # break
    try:
        with open(json_path, "r") as f:
            data = f.read()
            parsed = json.loads(data)
#             обозначаем имя изображения
        img_path = json_path.replace(json_path.split('/')[5], json_path.split('/')[5].split('_')[0])[:-5]
        im_yand = cv2.imread(img_path)
        for coord, str_ocr in word_info:
            # print(coord)
            vis_yandex = cv2.rectangle(im_yand, (coord[0], coord[1]), (coord[2], coord[3]), (255, 0, 255), 2)
        i = img_path.split('/')
        # print(i)
        # plt.imsave(f'img_examples_log/err_yand_imgs/{i[4]}__{i[5]}__{i[6]}', vis_yandex)
        print('OK')    
    except Exception as err:
        print(err)       